In [1]:

import os
import time
import math
import json
import pickle

import statistics

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data
import torch.nn.functional as F
import matplotlib
import matplotlib.pyplot as plt

from os import listdir
from os.path import join
from os.path import isfile

from tqdm import tqdm

from itertools import combinations

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import datasets
from sklearn import svm
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score


from libsvm.svmutil import svm_problem
from libsvm.svmutil import svm_parameter
from libsvm.svmutil import svm_train
from libsvm.svmutil import svm_predict
from libsvm.svmutil import evaluations

from sklearn.datasets import make_classification
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier

from ReliefF import ReliefF

from Function import svm_function

In [4]:
# save model
dir_path = "data/merge_data/0916chiu/classifier/output/"
onlyfiles = [f for f in listdir(dir_path) if isfile(join(dir_path, f))]

s = ""

for file in onlyfiles:
    if not ".dvc" in file and ("__esvm_s10_p10_e512_f10__v230106.json" in file or "__v230118.json" in file) :
        f = open(dir_path + file)
        json_dcit = json.load(f)
        f.close()
        name = file[:-len("__esvm_s10_p10_e512_f10__v230106.json")]
        print(name)
        name_path = "/data/jand/sixthTool/data/merge_data/0916chiu/classifier/"
        s += "nohup python save_model.py -m esvm -x %s -y %s -i %s -l %s -p %s -s %s" %(
            name_path + name + "_train.npy",
            name_path + name + "_train_y.npy",
            name_path + name + "_test.npy",
            name_path + name + "_test_y.npy",
            dir_path + file,
            "/data/jand/sixthTool/data/model/merge_data/0916chiu/classifier/" + file[:-5] + ".pickle &> data/log/save_model__" + file[:-5] + ".txt &\n")

nuc_lncRNA__cyto_lncRNA
nuc_lncRNA__cyto_mRNA
nuc_lncRNA__cyto_lncRNA__cyto_mRNA


In [14]:
with open("svm.sh", "w") as fp:
    fp.write(s)

In [ ]:
dir_path = "data/merge_data/0916chiu/classifier/output/nuc_lncRNA__cyto_lncRNA_esvmf10s50p4e512_v4"
# hp_df = pd.read_csv(dir_path)

with open(dir_path + ".json") as f:
    json_dict = json.load(f)
# print(file_str, test_roc_score, acc, recall, prec, spec, npv, f1sc)
print(json_dict["avg AUROC"], 
      json_dict["avg Accy"], 
      json_dict["avg Recall"], 
      json_dict["avg Prec"], 
      json_dict["avg Spec"], 
      json_dict["avg Npv"],
      json_dict["avg F1sc"])

In [49]:
dir_path = "data/model/merge_data/0916chiu/classifier/"
onlyfiles = [f for f in listdir(dir_path) if isfile(join(dir_path, f))]

lncrna_perf_list = []
mrna_perf_list = []

for file_name in onlyfiles:
    if not ".dvc" in file_name and not ".git" in file_name and "nuc_lncRNA__cyto_lncRNA__esvm_s10_p10_e512_f10__v230106" in file_name:
        file_str = file_name[:-len("__esvm_s10_p10_e512_f10__v230106.pickle")]
        print(file_str, file_name)

        with open(dir_path + file_name, 'rb') as f:
            clf = pickle.load(f)
        
        data_path = "data/merge_data/0916chiu/classifier/"
        test_x = np.load(data_path + "nuc_lncRNA__cyto_lncRNA" + "_test.npy")
        test_y = np.load(data_path + "nuc_lncRNA__cyto_lncRNA" + "_test_y.npy")
        pred_y, decision_values = clf.predict(test_x)
        # decision_values = np.where(np.isfinite(decision_values), decision_values, 0) 
        test_roc_score = metrics.roc_auc_score(test_y, decision_values)
        
        tn, fp, fn, tp = confusion_matrix(test_y, pred_y).ravel()
        
        acc = (tn + tp) / (tn + fp + fn + tp)
        recall = tp / (fn + tp)
        prec = (tp / (fp + tp))
        spec = (tn / (tn + fp))
        npv = (tn / (tn + fn))
        f1sc = (2 * (tp / (fn + tp)) * (tp / (fp + tp)) / ((tp / (fn + tp)) + (tp / (fp + tp))))
        
        lncrna_perf_list.append([file_str, test_roc_score, acc, recall, prec, spec, npv, f1sc])
        print("lncRAN test set AUROC: %.2f" % (test_roc_score))
        
        test_x = np.load(data_path + "nuc_mRNA__cyto_mRNA" + "_test.npy")
        test_y = np.load(data_path + "nuc_mRNA__cyto_mRNA" + "_test_y.npy")
        
        pred_y, decision_values = clf.predict(test_x)
        # decision_values = np.where(np.isfinite(decision_values), decision_values, 0) 
        test_roc_score = metrics.roc_auc_score(test_y, decision_values)
        
        tn, fp, fn, tp = confusion_matrix(test_y, pred_y).ravel()
        
        acc = (tn + tp) / (tn + fp + fn + tp)
        recall = tp / (fn + tp)
        prec = (tp / (fp + tp))
        spec = (tn / (tn + fp))
        npv = (tn / (tn + fn))
        f1sc = (2 * (tp / (fn + tp)) * (tp / (fp + tp)) / ((tp / (fn + tp)) + (tp / (fp + tp))))
        
        mrna_perf_list.append([file_str, test_roc_score, acc, recall, prec, spec, npv, f1sc])
        print("mRAN test set AUROC: %.2f" % (test_roc_score))
        

nuc_lncRNA__cyto_lncRNA nuc_lncRNA__cyto_lncRNA__esvm_s10_p10_e512_f10__v230106.pickle
lncRAN test set AUROC: 0.70
mRAN test set AUROC: 0.61


In [3]:
indep_list = [["nuc_mRNA__cyto_mRNA", "nuc_lncRNA__cyto_lncRNA"],
              ["nuc_lncRNA__cyto_mRNA", "nuc_mRNA__cyto_lncRNA"],
              ["nuc_lncRNA__cyto_lncRNA", "nuc_mRNA__cyto_mRNA"],
              ["nuc_lncRNA__cyto_lncRNA__cyto_mRNA", "nuc_mRNA"],
              ["nuc_lncRNA__nuc_mRNA__cyto_mRNA", "cyto_lncRNA"],
              ]
indep_perf_list = []
for test_str in indep_list:
    print(test_str[0], test_str[1])
    file_str = test_str[1]
    if os.path.exists(dir_path + test_str[0] + "__esvm_s10_p10_e512_f10__v230106.pickle"):
        with open(dir_path + test_str[0] + "__esvm_s10_p10_e512_f10__v230106.pickle", 'rb') as f:
            clf = pickle.load(f)
        
        data_path = "data/merge_data/0916chiu/classifier/"
        test_x = np.load(data_path + file_str + "_test.npy")
        test_y = np.load(data_path + file_str + "_test_y.npy")
        
        pred_y, decision_values = clf.predict(test_x)
        # decision_values = np.where(np.isfinite(decision_values), decision_values, 0) 
        if "nuc_mRNA" == file_str or "cyto_lncRNA" == file_str:
            test_roc_score = metrics.accuracy_score(test_y, pred_y)
        else:
            test_roc_score = metrics.roc_auc_score(test_y, decision_values)
        print("AUROC: %.2f" % (test_roc_score))
        tn, fp, fn, tp = confusion_matrix(test_y, pred_y).ravel()
            
        acc = (tn + tp) / (tn + fp + fn + tp)
        recall = tp / (fn + tp)
        prec = (tp / (fp + tp))
        spec = (tn / (tn + fp))
        npv = (tn / (tn + fn))
        f1sc = (2 * (tp / (fn + tp)) * (tp / (fp + tp)) / ((tp / (fn + tp)) + (tp / (fp + tp))))
        
        indep_perf_list.append([test_str[0], file_str, test_roc_score, acc, recall, prec, spec, npv, f1sc])

nuc_mRNA__cyto_mRNA nuc_lncRNA__cyto_lncRNA
nuc_lncRNA__cyto_mRNA nuc_mRNA__cyto_lncRNA
AUROC: 0.12
nuc_lncRNA__cyto_lncRNA nuc_mRNA__cyto_mRNA
AUROC: 0.61
nuc_lncRNA__cyto_lncRNA__cyto_mRNA nuc_mRNA
AUROC: 0.18
nuc_lncRNA__nuc_mRNA__cyto_mRNA cyto_lncRNA


/home/jand/.conda/envs/jand_venv/lib/python3.6/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in long_scalars


In [31]:
data_path = "data/merge_data/0916chiu/classifier/"
file_str = "nuc_lncRNA__cyto_lncRNA"
test_x = np.load(data_path + file_str + "_test.npy")
test_y = np.load(data_path + file_str + "_test_y.npy")

with open("data/model/merge_data/0916chiu/classifier/nuc_lncRNA__cyto_lncRNA__esvm_s10_p10_e512_f10__v230106.pickle", 'rb') as f:
    lncrna_clf = pickle.load(f)

with open("data/model/merge_data/0916chiu/classifier/nuc_mRNA__cyto_mRNA__svm.pickle", 'rb') as f:
    mrna_clf = pickle.load(f)

lnc_pred_y, lnc_pred_pro = lncrna_clf.predict(test_x)
m_pred_y = mrna_clf.predict(test_x)

In [32]:
m_pred_pro = mrna_clf.predict_proba(test_x)[:, 1]

In [4]:
def _perf_list(name, y, pred_y, dv):
    test_roc_score = metrics.roc_auc_score(y, dv)
    tn, fp, fn, tp = confusion_matrix(y, pred_y).ravel()

    acc = (tn + tp) / (tn + fp + fn + tp)
    recall = tp / (fn + tp)
    prec = (tp / (fp + tp))
    spec = (tn / (tn + fp))
    npv = (tn / (tn + fn))
    f1sc = (2 * (tp / (fn + tp)) * (tp / (fp + tp)) / ((tp / (fn + tp)) + (tp / (fp + tp))))
    return [name, test_roc_score, acc, recall, prec, spec, npv, f1sc]

In [47]:
perf_list = []
file_str = "lncRNA and mRNA"
pred_y = np.logical_and(lnc_pred_y, m_pred_y)
decision_values = np.where(pred_y, np.max(np.array([lnc_pred_pro, m_pred_pro]), axis=0), np.min(np.array([lnc_pred_pro, m_pred_pro]), axis=0))
perf_list.append(_perf_list(file_str, test_y, pred_y, decision_values))

file_str = "lncRNA or mRNA"
pred_y = np.logical_or(lnc_pred_y, m_pred_y)
decision_values = np.where(pred_y, np.max(np.array([lnc_pred_pro, m_pred_pro]), axis=0), np.min(np.array([lnc_pred_pro, m_pred_pro]), axis=0))
perf_list.append(_perf_list(file_str, test_y, pred_y, decision_values))

file_str = "lncRNA, mRNA high"
pred_y = np.where(abs(np.array(lnc_pred_pro) - 0.5) > abs(np.array(m_pred_pro) - 0.5), lnc_pred_y, m_pred_y)
decision_values = np.where(abs(np.array(lnc_pred_pro) - 0.5) > abs(np.array(m_pred_pro) - 0.5), lnc_pred_pro, m_pred_pro)
perf_list.append(_perf_list(file_str, test_y, pred_y, decision_values))

file_str = "lncRNA nuc, lncRNA, mRNA high"
pred_y = np.where((abs(np.array(lnc_pred_pro) - 0.5) > abs(np.array(m_pred_pro) - 0.5)) | np.array(lnc_pred_y, dtype=bool), lnc_pred_y, m_pred_y)
decision_values = np.where((abs(np.array(lnc_pred_pro) - 0.5) > abs(np.array(m_pred_pro) - 0.5)) | np.array(lnc_pred_y, dtype=bool), lnc_pred_pro, m_pred_pro)
perf_list.append(_perf_list(file_str, test_y, pred_y, decision_values))


In [48]:
pd.DataFrame(perf_list, columns=["name", "AUROC", "acc", "recall", "prec", "spec", "npv", "f1sc"]).to_csv("data/model/output/lncRNA_mRNA__230220_v2.csv", index=False)

In [2]:
def lnc_and_m(lnc_pred_y, lnc_pred_pro, m_pred_y, m_pred_pro):
    pred_y = np.logical_and(lnc_pred_y, m_pred_y)
    decision_values = np.where(pred_y, np.max(np.array([lnc_pred_pro, m_pred_pro]), axis=0), np.min(np.array([lnc_pred_pro, m_pred_pro]), axis=0))
    return pred_y, decision_values

def lnc_or_m(lnc_pred_y, lnc_pred_pro, m_pred_y, m_pred_pro):
    pred_y = np.logical_or(lnc_pred_y, m_pred_y)
    decision_values = np.where(pred_y, np.max(np.array([lnc_pred_pro, m_pred_pro]), axis=0), np.min(np.array([lnc_pred_pro, m_pred_pro]), axis=0))
    return pred_y, decision_values

def lnc_high_m(lnc_pred_y, lnc_pred_pro, m_pred_y, m_pred_pro):
    pred_y = np.where(abs(np.array(lnc_pred_pro) - 0.5) > abs(np.array(m_pred_pro) - 0.5), lnc_pred_y, m_pred_y)
    decision_values = np.where(abs(np.array(lnc_pred_pro) - 0.5) > abs(np.array(m_pred_pro) - 0.5), lnc_pred_pro, m_pred_pro)
    return pred_y, decision_values

def lnc_n_lnc_high_m(lnc_pred_y, lnc_pred_pro, m_pred_y, m_pred_pro):
    pred_y = np.where((abs(np.array(lnc_pred_pro) - 0.5) > abs(np.array(m_pred_pro) - 0.5)) | np.array(lnc_pred_y, dtype=bool), lnc_pred_y, m_pred_y)
    decision_values = np.where((abs(np.array(lnc_pred_pro) - 0.5) > abs(np.array(m_pred_pro) - 0.5)) | np.array(lnc_pred_y, dtype=bool), lnc_pred_pro, m_pred_pro)
    return pred_y, decision_values

In [5]:
lncrna_perf_list = []
mrna_perf_list = []

with open("data/model/merge_data/0916chiu/classifier/nuc_lncRNA__cyto_lncRNA__esvm_s10_p10_e512_f10__v230106.pickle", 'rb') as f:
    lncrna_clf = pickle.load(f)

with open("data/model/merge_data/0916chiu/classifier/nuc_mRNA__cyto_mRNA__svm.pickle", 'rb') as f:
    mrna_clf = pickle.load(f)

# lncRNA data test
data_path = "data/merge_data/0916chiu/classifier/"
file_str = "nuc_lncRNA__cyto_lncRNA"
test_x = np.load(data_path + file_str + "_test.npy")
test_y = np.load(data_path + file_str + "_test_y.npy")

# lncRNA, mRNA model pred
lnc_pred_y, lnc_pred_pro = lncrna_clf.predict(test_x)
m_pred_y = mrna_clf.predict(test_x)
m_pred_pro = mrna_clf.predict_proba(test_x)[:, 1]

# lncRNA
model_str = "lncRNA"
lncrna_perf_list.append(_perf_list(model_str, test_y, lnc_pred_y, lnc_pred_pro))

# lncRNA and mRNA
pred_y, decision_values = lnc_and_m(lnc_pred_y, lnc_pred_pro, m_pred_y, m_pred_pro)
lncrna_perf_list.append(_perf_list("lncRNA and mRNA", test_y, pred_y, decision_values))
# lncRNA or mRNA 
pred_y, decision_values = lnc_or_m(lnc_pred_y, lnc_pred_pro, m_pred_y, m_pred_pro)
lncrna_perf_list.append(_perf_list("lncRNA or mRNA", test_y, pred_y, decision_values))
# lncRNA high mRNA
pred_y, decision_values = lnc_high_m(lnc_pred_y, lnc_pred_pro, m_pred_y, m_pred_pro)
lncrna_perf_list.append(_perf_list("lncRNA high mRNA", test_y, pred_y, decision_values))
# lncRNA nuc, lncRNA high mRNA
pred_y, decision_values = lnc_n_lnc_high_m(lnc_pred_y, lnc_pred_pro, m_pred_y, m_pred_pro)
lncrna_perf_list.append(_perf_list("lncRNA nuc, lncRNA high mRNA", test_y, pred_y, decision_values))


# mRNA data test
data_path = "data/merge_data/0916chiu/classifier/"
file_str = "nuc_mRNA__cyto_mRNA"
test_x = np.load(data_path + file_str + "_test.npy")
test_y = np.load(data_path + file_str + "_test_y.npy")

# lncRNA, mRNA model pred
lnc_pred_y, lnc_pred_pro = lncrna_clf.predict(test_x)
m_pred_y = mrna_clf.predict(test_x)
m_pred_pro = mrna_clf.predict_proba(test_x)[:, 1]

# lncRNA
model_str = "lncRNA"
mrna_perf_list.append(_perf_list(model_str, test_y, lnc_pred_y, lnc_pred_pro))

# lncRNA and mRNA
pred_y, decision_values = lnc_and_m(lnc_pred_y, lnc_pred_pro, m_pred_y, m_pred_pro)
mrna_perf_list.append(_perf_list("lncRNA and mRNA", test_y, pred_y, decision_values))
# lncRNA or mRNA 
pred_y, decision_values = lnc_or_m(lnc_pred_y, lnc_pred_pro, m_pred_y, m_pred_pro)
mrna_perf_list.append(_perf_list("lncRNA or mRNA", test_y, pred_y, decision_values))
# lncRNA high mRNA
pred_y, decision_values = lnc_high_m(lnc_pred_y, lnc_pred_pro, m_pred_y, m_pred_pro)
mrna_perf_list.append(_perf_list("lncRNA high mRNA", test_y, pred_y, decision_values))
# lncRNA nuc, lncRNA high mRNA
pred_y, decision_values = lnc_n_lnc_high_m(lnc_pred_y, lnc_pred_pro, m_pred_y, m_pred_pro)
mrna_perf_list.append(_perf_list("lncRNA nuc, lncRNA high mRNA", test_y, pred_y, decision_values))

In [6]:
print(lncrna_perf_list)
print(mrna_perf_list)

[['lncRNA', 0.7036889172788247, 0.6649029982363316, 0.6752475247524753, 0.9291553133514986, 0.5806451612903226, 0.18, 0.7821100917431192], ['lncRNA and mRNA', 0.6846055573299265, 0.3968253968253968, 0.3405940594059406, 0.9502762430939227, 0.8548387096774194, 0.13730569948186527, 0.5014577259475219], ['lncRNA or mRNA', 0.6825614819546472, 0.7354497354497355, 0.7762376237623763, 0.9137529137529138, 0.4032258064516129, 0.18115942028985507, 0.8394004282655247], ['lncRNA high mRNA', 0.7156978601085915, 0.691358024691358, 0.695049504950495, 0.9435483870967742, 0.6612903225806451, 0.21025641025641026, 0.8004561003420751], ['lncRNA nuc, lncRNA high mRNA', 0.7049185563717661, 0.6984126984126984, 0.7168316831683168, 0.9282051282051282, 0.5483870967741935, 0.192090395480226, 0.8089385474860334]]
[['lncRNA', 0.6139720144084234, 0.613884926295768, 0.7290108063175395, 0.6434336023477623, 0.46, 0.5594594594594594, 0.6835541699142633], ['lncRNA and mRNA', 0.6723866260275237, 0.6329053732762719, 0.5295

In [7]:
lncrna_perf_df = pd.DataFrame(lncrna_perf_list, columns=["name", "lncRNA AUROC", "lncRNA acc", "lncRNA recall", "lncRNA prec", "lncRNA spec", "lncRNA npv", "lncRNA f1sc"])
mrna_perf_df = pd.DataFrame(mrna_perf_list, columns=["name", "mRNA AUROC", "mRNA acc", "mRNA recall", "mRNA prec", "mRNA spec", "mRNA npv", "mRNA f1sc"])

In [9]:
pd.merge(lncrna_perf_df, mrna_perf_df, on="name", how="inner").to_csv("data/model/output/test_lncRNA_mRNA__230306.csv", index=False)

In [29]:
file_str = "nuc_lncRNA__cyto_lncRNA + nuc_mRNA__cyto_mRNA"
# decision_values = np.logical_and(lnc_pred_y, m_pred_y)
# decision_values = np.logical_or(lnc_pred_y, m_pred_y)
decision_values = np.where(abs(np.array(lnc_pred_pro) - 0.5) > abs(np.array(m_pred_pro) - 0.5), lnc_pred_y, m_pred_y)
# decision_values = np.where((abs(np.array(lnc_pred_pro) - 0.5) > abs(np.array(m_pred_pro) - 0.5)) | np.array(lnc_pred_y, dtype=bool), lnc_pred_y, m_pred_y)
test_roc_score = metrics.roc_auc_score(test_y, decision_values)
        
tn, fp, fn, tp = confusion_matrix(test_y, decision_values).ravel()

acc = (tn + tp) / (tn + fp + fn + tp)
recall = tp / (fn + tp)
prec = (tp / (fp + tp))
spec = (tn / (tn + fp))
npv = (tn / (tn + fn))
f1sc = (2 * (tp / (fn + tp)) * (tp / (fp + tp)) / ((tp / (fn + tp)) + (tp / (fp + tp))))
print(file_str, test_roc_score, acc, recall, prec, spec, npv, f1sc)
print("AUROC: %.2f" % (test_roc_score))

nuc_lncRNA__cyto_lncRNA + nuc_mRNA__cyto_mRNA 0.6781699137655702 0.691358024691358 0.695049504950495 0.9435483870967742 0.6612903225806451 0.21025641025641026 0.8004561003420751
AUROC: 0.68
